In [2]:
import tensorflow as tf
from tensorflow import keras
import numpy as np

In [5]:
f = open('training_data.txt', 'r')
text = f.read()
text[:2000]

"Iran, the Number One State of Sponsored Terror with numerous violations of Human Rights occurring on an hourly basis, has now closed down the Internet so that peaceful demonstrators cannot communicate. Not good! HAPPY NEW YEAR! We are MAKING AMERICA GREAT AGAIN, and much faster than anyone thought possible! My deepest condolences to the victims of the terrible shooting in Douglas County @DCSheriff, and their families. We love our police and law enforcement  God Bless them all! #LESM As our Country rapidly grows stronger and smarter, I want to wish all of my friends, supporters, enemies, haters, and even the very dishonest Fake News Media, a Happy and Healthy New Year. 2018 will be a great year for America! What a year its been, and we're just getting started. Together, we are MAKING AMERICA GREAT AGAIN! Happy New Year!! WOW, @foxandfrlends  Dossier is bogus. Clinton Campaign, DNC funded Dossier. FBI CANNOT after all of this time VERIFY CLAIMS IN DOSSIER OF RUSSIA/TRUMP COLLUSION. FBI 

In [6]:
tokenizer = keras.preprocessing.text.Tokenizer(char_level=True, lower=False)
tokenizer.fit_on_texts(text)
tokenizer.word_index

{' ': 1,
 'e': 2,
 't': 3,
 'a': 4,
 'o': 5,
 'n': 6,
 'i': 7,
 'r': 8,
 's': 9,
 'l': 10,
 'h': 11,
 'd': 12,
 'u': 13,
 'm': 14,
 'c': 15,
 'g': 16,
 'y': 17,
 'p': 18,
 '.': 19,
 'w': 20,
 'f': 21,
 'b': 22,
 'A': 23,
 'v': 24,
 ',': 25,
 'k': 26,
 'S': 27,
 'T': 28,
 '!': 29,
 'I': 30,
 'C': 31,
 'R': 32,
 'N': 33,
 'E': 34,
 'M': 35,
 'W': 36,
 'O': 37,
 'P': 38,
 'F': 39,
 'H': 40,
 'G': 41,
 'D': 42,
 'B': 43,
 'L': 44,
 'U': 45,
 '@': 46,
 'x': 47,
 '0': 48,
 '#': 49,
 'J': 50,
 'K': 51,
 'j': 52,
 '"': 53,
 "'": 54,
 'V': 55,
 '1': 56,
 'Y': 57,
 '2': 58,
 ':': 59,
 'z': 60,
 '?': 61,
 '7': 62,
 '/': 63,
 '5': 64,
 '3': 65,
 '4': 66,
 '6': 67,
 'q': 68,
 'X': 69,
 '8': 70,
 '9': 71,
 'Q': 72,
 'Z': 73}

In [7]:
max_id = len(tokenizer.word_index)
dataset_size = tokenizer.document_count
print(max_id, dataset_size)

73 281143


In [8]:
[seq] = np.array(tokenizer.texts_to_sequences([text])) - 1

In [9]:
seq[:100]

array([29,  7,  3,  5, 24,  0,  2, 10,  1,  0, 32, 12, 13, 21,  1,  7,  0,
       36,  5,  1,  0, 26,  2,  3,  2,  1,  0,  4, 20,  0, 26, 17,  4,  5,
        8,  4,  7,  1, 11,  0, 27,  1,  7,  7,  4,  7,  0, 19,  6,  2, 10,
        0,  5, 12, 13,  1,  7,  4, 12,  8,  0, 23,  6,  4,  9,  3,  2,  6,
        4,  5,  8,  0,  4, 20,  0, 39, 12, 13,  3,  5,  0, 31,  6, 15, 10,
        2,  8,  0,  4, 14, 14, 12,  7,  7,  6,  5, 15,  0,  4,  5])

In [24]:
train_size = .1
seq = seq[:int(train_size * dataset_size)]

In [25]:
from sklearn.model_selection import train_test_split
train, val = train_test_split(seq, test_size=0.1, shuffle=False)

In [26]:
len(train)+len(val)

28114

In [27]:
n_steps = 100
window_length = n_steps + 1
batch_size = 32

def prepare_ds(ds):
    # packages the list of tokens into data sets of length 101 
    ds = ds.window(window_length, shift=1, drop_remainder=True)
    ds = ds.flat_map(lambda window: window.batch(window_length))
    ds = ds.shuffle(dataset_size).batch(batch_size)
    ds = ds.map(lambda windows: (windows[:, :-1], windows[:, 1:]))
    ds = ds.map(lambda X_batch, Y_batch: (tf.one_hot(X_batch, depth=max_id), Y_batch))
    ds = ds.prefetch(1)
    return ds

In [28]:
dataset_train = prepare_ds(tf.data.Dataset.from_tensor_slices(train))
dataset_val = prepare_ds(tf.data.Dataset.from_tensor_slices(val))

In [29]:
for i, element in enumerate(dataset_train):
    if i >= 1:
        break
    print(element)

(<tf.Tensor: shape=(32, 100, 73), dtype=float32, numpy=
array([[[0., 0., 0., ..., 0., 0., 0.],
        [0., 1., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       [[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 1., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]],

       ...,

       [[1., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [1., 0.

In [100]:
model = keras.models.Sequential([
    keras.layers.GRU(100, return_sequences=True, input_shape=[None, max_id],
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.GRU(100, return_sequences=True,
                     dropout=0.2, recurrent_dropout=0.2),
    keras.layers.TimeDistributed(keras.layers.Dense(max_id, activation="softmax"))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=['accuracy'])

In [101]:
early_stopping_cb = keras.callbacks.EarlyStopping(monitor='val_accuracy', patience=2, restore_best_weights=True)

In [102]:
history = model.fit(dataset_train,
                    epochs=20,
                    callbacks=early_stopping_cb,
                    validation_data=dataset_val
                    )

Epoch 1/20
788/788 [==============================] - 349s 443ms/step - loss: 2.5413 - accuracy: 0.3104 - val_loss: 2.1475 - val_accuracy: 0.4009
Epoch 2/20
788/788 [==============================] - 350s 445ms/step - loss: 1.9577 - accuracy: 0.4400 - val_loss: 1.9767 - val_accuracy: 0.4580
Epoch 3/20
788/788 [==============================] - 349s 443ms/step - loss: 1.7555 - accuracy: 0.4925 - val_loss: 1.9203 - val_accuracy: 0.4813
Epoch 4/20
788/788 [==============================] - 349s 443ms/step - loss: 1.6421 - accuracy: 0.5207 - val_loss: 1.9060 - val_accuracy: 0.4836
Epoch 5/20
788/788 [==============================] - 349s 443ms/step - loss: 1.5673 - accuracy: 0.5385 - val_loss: 1.8993 - val_accuracy: 0.4888
Epoch 6/20
788/788 [==============================] - 349s 443ms/step - loss: 1.5120 - accuracy: 0.5521 - val_loss: 1.9093 - val_accuracy: 0.4918
Epoch 7/20
788/788 [==============================] - 349s 443ms/step - loss: 1.4715 - accuracy: 0.5618 - val_loss: 1.9178 -

In [94]:
def preprocess(text):
    X = np.array(tokenizer.texts_to_sequences([text])) - 1
    return tf.one_hot(X, max_id)

In [103]:
X_test = preprocess("Make Americ")
proba = model.predict(X_test)[0, -1:, :]
np.around(proba, 4)

array([[3.000e-04, 2.000e-04, 8.000e-04, 9.961e-01, 7.000e-04, 1.000e-04,
        4.000e-04, 0.000e+00, 2.000e-04, 1.000e-04, 4.000e-04, 0.000e+00,
        1.000e-04, 0.000e+00, 1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00,
        2.000e-04, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 1.000e-04, 0.000e+00, 0.000e+00, 1.000e-04, 0.000e+00,
        1.000e-04, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00, 0.000e+00,
        0.000e+00]], dtype=float32)

In [104]:
Y_pred = np.argmax(model.predict(X_test), axis=-1)
tokenizer.sequences_to_texts(Y_pred + 1)[0][-1]

'a'

In [97]:
def next_char(text, temperature=1):
    X_new = preprocess(text)
    y_proba = model.predict(X_new)[0, -1:, :]
    rescaled_logits = tf.math.log(y_proba) / temperature
    char_id = tf.random.categorical(rescaled_logits, num_samples=1) + 1
    return tokenizer.sequences_to_texts(char_id.numpy())[0]


def complete_text(text, n_chars=10, temperature=1):
    for _ in range(n_chars):
        text += next_char(text, temperature)
    return text

In [118]:
complete_text("T", n_chars=140, temperature=.1)

'T TRUSTED NEWS! We are despent to the record and the record and the same and the same and the same and the record and the Fake News Media is '

In [119]:
complete_text("T", n_chars=140, temperature=.5)

'T MAKE AMERICA GREAT AGAIN! LAST to the Iran most me this year for the very was my going of the Fake News Media is to the saying Harry and wh'

In [121]:
complete_text("T", n_chars=140, temperature=1)

'Time asked to, buch Americans comons. Merry McCabe, that is the @Whitej the encto be the people thouse in hesters whent themselves will be th'

In [120]:
complete_text("T", n_chars=140, temperature=1.5)

'Take New Brks USA. I alsoeding wimer the dayncuration, a5 a1dastrek, you paoted hers, fartitial an Dor deeired talkh Korea. I hreod to forhyo'